In [26]:
import pandas as pd
import googlemaps
import json
from haversine import haversine
from sklearn import preprocessing
import folium

In [27]:
#데이터프레임 불러오기
df = pd.read_csv("꽉찬전국도서관표준데이터-20201223.csv", index_col=0, encoding='utf-8')
columns = ['도서관명', '도서관유형', '열람좌석수', '자료수', '건물면적', '위도', '경도', '좌표']
df = df[columns]

In [28]:
choice_ex = ["열람좌석수", "건물면적", "자료수"]

def user_preference():
    
    while len(choice_ex) == 3:
        print("-----아래의 목록에서 골라주세요-----") 
        print(choice_ex)
        print("------------------------------------")
        point1 = input("1순위 : ")
        if point1 not in choice_ex:
            print("입력이 잘못되었습니다. 다시 입력해주세요")
            continue
        else:
            choice_ex.remove(point1)
            
    while len(choice_ex) == 2:
        print("-----아래의 목록에서 골라주세요-----") 
        print(choice_ex)
        print("------------------------------------")
        point2 = input("2순위 : ")
        if point2 not in choice_ex:
            print("입력이 잘못되었습니다. 다시 입력해주세요")
            continue
        else:
            choice_ex.remove(point2)
    
    point3 = choice_list[0]
                       
    print("3순위 : " + point3)
    return point1 , point2 , point3

In [29]:
point1 , point2 , point3 = user_preference()

loc = input("위치 입력 : ")

In [30]:
#'거리' 구하기 밑작업
df['거리'] = 0.00

#Google Map API 접속
gmaps_key = "##############API KEY#############"
gmaps = googlemaps.Client(key = gmaps_key)

#좌표값 받는 함수
def user_location(user_loc): 
    while True :
        tmp = gmaps.geocode(user_loc) #주소로 검색해 좌표로 돌려주는 API기능
        if bool(tmp) : #검색이 된다면
            tmp_loc = tmp[0].get('geometry')
            lat = tmp_loc['location']['lat']
            lng = tmp_loc['location']['lng']
            if ((lat > 0) & (lng > 0)): #검색성공
                return lat, lng
        else:
            print('정확한 도로명 주소를 남겨주세요')
            user_loc = input("위치 입력 : ")
            continue
            

In [31]:
#입력받은 위치 기반 '거리' 열 구하기
location = loc
loc1 = user_location(location)
df['거리'] = df.apply(lambda row: haversine(loc1, (row['위도'], row['경도'])), axis = 1)
#haversine 함수 재사용 #처리 시간을 줄이기 위한 apply 함수 활용

In [32]:
#가까운 도서관 상위 10개 뽑기
df_top = df.sort_values(by='거리', axis=0).head(10)
df_top

In [33]:
#정규화
org_columns = ['도서관명', '위도', '경도', '열람좌석수', '자료수'] #위도와 경도의 정규화방지를 위해 필요한 열만 넣어서 데이터프레임을 따로복사
df_org = df_top[org_columns]
columns = ['도서관명', '열람좌석수', '자료수', '건물면적', '거리']
df_top = df_top[columns]

df_top.set_index('도서관명', inplace=True) #두개 다 인덱스를 도서관명으로 지정
df_org.set_index('도서관명', inplace=True)

#정규화를 위해 데이터프레임 잠시 분리
df_top_dist = df_top['거리']

df_top_nodist = df_top.copy(deep=True)
del df_top_nodist['거리']

In [34]:
# Min-Max 정규화
scaler = preprocessing.MinMaxScaler()
scaler.fit(df_top_nodist)
df_top_norm = scaler.transform(df_top_nodist)

# 정규화값 결정하기
df_top_norm = pd.DataFrame(df_top_norm)
df_top_norm.index = df_top_nodist.index
df_top_norm.columns = df_top_nodist.columns
df_top_norm['거리'] = df_top_dist
df_top_norm

In [35]:
#점수 내기
df_top_norm['score'] = (df_top_norm[point1] * 6) \
                        + (df_top_norm[point2] * 3) \
                        + (df_top_norm[point3] * 1)
df_top_norm['score'] = df_top_norm['score']
df_top_norm

In [36]:
#현재까지는 정렬구조가 같으므로 기존 복사해둔 df_org에서 위도와 경도 열을 가져와서 그대로 넣어줌(score 기준으로 정렬하기전에) 
for name in df_org:
    df_top_norm["위도"] = df_org["위도"]
    df_top_norm["경도"] = df_org["경도"]
    df_top_norm["자료수(원본)"] = df_org["자료수"]
    df_top_norm["열람좌석수(원본)"] = df_org["열람좌석수"]

In [37]:
#최종 데이터프레임
df_top_norm = df_top_norm.sort_values(by="score",ascending=False)
df_top_norm

In [38]:
#지도 시각화

n_list = []
m_number_list = []
s_number_list = []


map = folium.Map(location=loc1, zoom_start=14)


for i , n in enumerate(df_top_norm.index):
    n_list.append(n)
    m_number_list.append(df_top_norm['자료수(원본)'][n])
    s_number_list.append(df_top_norm['열람좌석수(원본)'][n])
    
    if i == 0 : 
        folium.Marker([df_top_norm['위도'][n], df_top_norm['경도'][n]], 
                  popup = "추천순위 : " + str(i+1)
                  + "<br>도서관명 : " + n
                  + "<br>자료수 : " + str(df_top_norm['자료수(원본)'][n]) 
                  + "<br>열람좌석수 : " + str(df_top_norm['열람좌석수(원본)'][n]), 
                  icon= (folium.Icon(icon="book", prefix="fa", color="red"))).add_to(map)
    
    elif i <= 3 :
        folium.Marker([df_top_norm['위도'][n], df_top_norm['경도'][n]], 
                  popup = "추천순위 : " + str(i+1)
                  + "<br>도서관명 : " + n
                  + "<br>자료수 : " + str(df_top_norm['자료수(원본)'][n]) 
                  + "<br>열람좌석수 : " + str(df_top_norm['열람좌석수(원본)'][n]), 
                  icon= (folium.Icon(icon="book", prefix="fa", color="blue"))).add_to(map)
        
    else :
        folium.Marker([df_top_norm['위도'][n], df_top_norm['경도'][n]], 
                  popup = "추천순위 : " + str(i+1)
                  + "<br>도서관명 : " + n
                  + "<br>자료수 : " + str(df_top_norm['자료수(원본)'][n]) 
                  + "<br>열람좌석수 : " + str(df_top_norm['열람좌석수(원본)'][n]), 
                  icon= (folium.Icon(icon="book", prefix="fa", color="green"))).add_to(map)
                  
    
    map.save('map.html')

In [39]:
#문구 삽입

print("가장 추천하는 도서관 : {}".format(df_top_norm.index[0]))

print("지도를 확인하세요.")

In [40]:
import webbrowser
import os , time
print("5초 후 map이 자동으로 열립니다")

for x in reversed(range(5)):
    print(x + 1)
    time.sleep(1)
webbrowser.open('file://' + os.path.realpath("./map.html"))

### 목차

1. 데이터 셋 설명

2. 전처리
  - 결측치처리
  - 정규화
  
3. 순위도출
  - 위치입력
  - 거리계산
  - 선호도입력
  - 점수계산

4. 시각화
  - 지도시각화
  - 순위별 분류